<a href="https://colab.research.google.com/github/Reynxzz/capstone-ipb/blob/main/Capstone_IPB_Making_Dataset_%26_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Making Dataset (place, lat, lon, elevation)

Ngambil data elevation dari opentopodata API

STEP bikin dataset:
1. Bikin random point di google earth sekitaran IPB
2. Export latitude dan longitude dari point2 itu
3. Ambil data elevation per point tersebut pake Open Topodata API
4. Generate random route (cyclic) dengan start Agrimart dan Rektorat
5. Sesuaikan distancenya (max 3 km dan max 5 km)
6. Sesuaikan kesulitannya pake data elevasi (Easy, Medium, Hard) --> lagi disini
7. Sesuaikan dengan pemandangan

Import CSV datasets from Google Earth

In [ ]:
import csv

# Open the CSV file for reading
with open('/content/drive/My Drive/Capstone IPB/Tempat_di_IPB.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)

    # Create a list of tuples containing latitude and longitude
    coordinates = [(float(row['Y']), float(row['X'])) for row in reader]

Get Elevation

In [ ]:
import requests

def get_elevations(coordinates):
    """Lihat dokumentasi disini bro https://www.opentopodata.org/"""

    # Build the API URL for the OpenTopo API
    url = "https://api.opentopodata.org/v1/aster30m"

    # Make the API call for each coordinate and extract the elevation data
    elevations = []
    for coord in coordinates:
        lat, lon = coord
        query = f"{url}?locations={lat},{lon}"
        response = requests.get(query)
        if response.status_code == 200:
            data = response.json()
            elevation = data["results"][0]["elevation"]
            elevations.append(elevation)
        else:
            elevations.append(None)

    # Return the list of elevations
    return elevations

In [ ]:
elevations = get_elevations(coordinates)

# Print the list of elevations
print(elevations)

[184.0, 180.0, 175.0, 176.0, None, 184.0, None, 167.0, None, None, 124.0, None, 165.0, None]


Generate Routes

Bikin rute random

In [ ]:
import random
import json
import folium

def routes_maker(total_point, max_distance):
    subset = []
    # define start and end coordinates
    start = coordinates[0]
    end = coordinates[0]

    # randomly select a subset of coordinates
    subset = random.sample(coordinates, k=total_point)  # k is the number of coordinates in the subset

    # add the start and end coordinates to the beginning and end of the subset
    subset.insert(0, start)
    subset.append(end)

    api_url = 'https://api.openrouteservice.org/v2/directions/foot-walking'
    api_key = '5b3ce3597851110001cf6248d4725ac662f8461eb589679758fb20ea'

    locations = [list(r) for r in subset]
    total_distance = 0

    for i in range(len(locations)-1):
        # Define the start and end coordinates for the API call
        start_coords = f"{locations[i][1]},{locations[i][0]}"
        end_coords = f"{locations[i+1][1]},{locations[i+1][0]}"
        
        # Define the API call parameters, including the start and end coordinates, the API key, and the travel mode (walking)
        params = {
            'api_key': api_key,
            'start': start_coords,
            'end': end_coords,
            'profile': 'foot-walking',
        }
        
        # Make the API call and parse the response as JSON
        response = requests.get(api_url, params=params)
        data = json.loads(response.text)
        
        # Extract the walking distance from the response and print it
        distance = data['features'][0]['properties']['segments'][0]['distance']
        print(f"Distance between locations {i+1} and {i+2}: {distance:.2f} meters")
        total_distance += distance
        
        # visualize route
        m = folium.Map(location=start, zoom_start=14)
        folium.PolyLine(locations=[list(reversed(coord)) 
                              for coord in 
                              data['features'][0]['geometry']['coordinates']]).add_to(m)

    print(f"total distance: {total_distance:.2f}")
    
    if total_distance > max_distance + 500:
          total_distance = routes_maker(total_point, max_distance)


    return total_distance, m 

In [ ]:
total_distances, m = routes_maker(2, 5000)

Distance between locations 1 and 2: 2041.40 meters
Distance between locations 2 and 3: 1239.00 meters
Distance between locations 3 and 4: 3071.90 meters
total distance: 6352.30
Distance between locations 1 and 2: 3071.90 meters
Distance between locations 2 and 3: 1239.00 meters
Distance between locations 3 and 4: 2041.40 meters
total distance: 6352.30
Distance between locations 1 and 2: 3071.90 meters
Distance between locations 2 and 3: 3071.90 meters
Distance between locations 3 and 4: 0.00 meters
total distance: 6143.80
Distance between locations 1 and 2: 0.00 meters
Distance between locations 2 and 3: 2665.60 meters
Distance between locations 3 and 4: 2665.60 meters
total distance: 5331.20


Visualisasi rute

In [ ]:
m